In [ ]:
pip install chart_studio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 1.2 MB/s eta 0:00:00


In [ ]:
##TESTING

### Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import figure
import plotly.graph_objects as go
import chart_studio
import chart_studio.plotly as py
import plotly.express as px

#Read the collected/cleaned NBA and college data
final_df= pd.read_csv('/content/Full College and Career Stats.csv')

#Plot distribtuion of WS
import chart_studio.plotly as py
import plotly.express as px
fig = px.histogram(final_df, x="WS",labels={'WS':'Win Shares','y':'count'})
fig.show()

# Drop other advanced stats and College Team
drop_cols=['WS/48', 'BPM', 'VORP', 'College']
final_df=final_df.drop(drop_cols,axis=1)
final_df = final_df.dropna(axis=0)

##Setting Player as the dataframe's index
final_df.set_index('Player')

#Split data into train and test by years
x_train = final_df[final_df['Draft_Year'].isin([2011,2012, 2013, 2014, 2015, 2016, 2017, 2018])]
x_test = final_df[final_df['Draft_Year'].isin([2019, 2020, 2021, 2022, 2023])]
x_train=x_train.set_index('Player')
x_test=x_test.set_index('Player')

#Create a x/y split for independent and dependent variables
y_train = x_train['WS']
y_test = x_test['WS']
x_train = x_train.drop(['Draft_Year','WS'],axis=1)
x_test = x_test.drop(['Draft_Year','WS'],axis=1)

### Defining a Random Forest model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import MinMaxScaler

#Initializing the Linear Regression model
rf = RandomForestRegressor()

#Fitting the training data
rf.fit(x_train,y_train)

#Predict on test data
y_pred = rf.predict(x_test)
y_pred_train = rf.predict(x_train)
train_r2 = r2_score(y_train, y_pred_train)

# Calculate R^2 value for the testing data
test_r2 = r2_score(y_test, y_pred)

print(f"Training R^2: {train_r2}")
print(f"Testing R^2: {test_r2}")


# Reset_indexes
x_test=x_test.reset_index()
x_train=x_train.reset_index()

#Create a Results dataframe for analysis
Results=pd.DataFrame()
Results['Player']=x_test['Player']
Results['WS_Pred']=y_pred
Results.set_index('Player')

#Bring back full dataset
Results=Results.merge(final_df,how='left')
#Results=Results.merge(draftdf,'left')


###Creating a visualization for the regression model
fig= go.Figure([
    go.Scatter(x=Results['Player'], y = Results['WS'], name='Win Shares',mode='markers'),
    go.Scatter(x=Results['Player'], y=Results['WS_Pred'], name='Predicted Win Shares')
])
fig.show()

#Slicing Results
Result_slice=Results[['Pk','Player','WS','WS_Pred','Draft_Year']].sort_values('WS_Pred',ascending=False)

#Performing steps to filter the data by years for further analysis
Result_slice['Deviation']=Result_slice['WS']-Result_slice['WS_Pred']
Result_slice.WS_Pred=Result_slice.WS_Pred.round(decimals=2)

#Filtering continued
Result_2019 = Result_slice[Result_slice.Draft_Year == 2019.0]
Result_2020 = Result_slice[Result_slice.Draft_Year == 2020.0]
Result_2021 = Result_slice[Result_slice.Draft_Year == 2021.0]
Result_2022 = Result_slice[Result_slice.Draft_Year == 2022.0]
Result_2023 = Result_slice[Result_slice.Draft_Year == 2023.0]

Result_2019['Original_Draft_Pick'] = Result_2019['Pk']
Result_2020['Original_Draft_Pick'] = Result_2020['Pk']
Result_2021['Original_Draft_Pick'] = Result_2021['Pk']
Result_2022['Original_Draft_Pick'] = Result_2022['Pk']
Result_2023['Original_Draft_Pick'] = Result_2023['Pk']

Result_2019['Rank_of_WS_Prediction'] = Result_2019['WS_Pred'].rank(method='dense', ascending=False).astype(int)
Result_2020['Rank_of_WS_Prediction'] = Result_2020['WS_Pred'].rank(method='dense', ascending=False).astype(int)
Result_2021['Rank_of_WS_Prediction'] = Result_2021['WS_Pred'].rank(method='dense', ascending=False).astype(int)
Result_2022['Rank_of_WS_Prediction'] = Result_2022['WS_Pred'].rank(method='dense', ascending=False).astype(int)
Result_2023['Rank_of_WS_Prediction'] = Result_2023['WS_Pred'].rank(method='dense', ascending=False).astype(int)

Result_2019['Rank_of_Current_WS'] = Result_2019['WS'].rank(method='dense', ascending=False).astype(int)
Result_2020['Rank_of_Current_WS'] = Result_2020['WS'].rank(method='dense', ascending=False).astype(int)
Result_2021['Rank_of_Current_WS'] = Result_2021['WS'].rank(method='dense', ascending=False).astype(int)
Result_2022['Rank_of_Current_WS'] = Result_2022['WS'].rank(method='dense', ascending=False).astype(int)
Result_2023['Rank_of_Current_WS'] = Result_2023['WS'].rank(method='dense', ascending=False).astype(int)

Result_2019['Current_WS'] = Result_2019['WS']
Result_2020['Current_WS'] = Result_2020['WS']
Result_2021['Current_WS'] = Result_2021['WS']
Result_2022['Current_WS'] = Result_2022['WS']
Result_2023['Current_WS'] = Result_2023['WS']

###Visualizations code for top 15 WS_Pred: Change Year of df name for year wanted to view. (2019)
Result_2019['Rank'] = Result_2019['WS_Pred'].rank(method='dense', ascending=False).astype(int)
colorscale = [[0, '#800000'],[.5, '#fffffe'],[1, '#ffffff']]
fig = ff.create_table(Result_2019[['Rank','Player','Original_Draft_Pick','WS_Pred','Rank_of_Current_WS']].sort_values('WS_Pred',ascending=False).head(15),colorscale)
for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i].font.size = 13
fig.show()

###Visualizations code for top 15 WS: Change Year of df name for year wanted to view. (2019)
Result_2019['Rank'] = Result_2019['WS'].rank(method='dense', ascending=False).astype(int)
colorscale = [[0, '#800000'],[.5, '#fffffe'],[1, '#ffffff']]
fig = ff.create_table(Result_2019[['Rank','Player','Original_Draft_Pick','Current_WS','Rank_of_WS_Prediction']].sort_values('Current_WS',ascending=False).head(15),colorscale)
for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i].font.size = 13
fig.show()

###Visualizations code for top 15 WS_Pred: Change Year of df name for year wanted to view. (2020)
Result_2020['Rank'] = Result_2020['WS_Pred'].rank(method='dense', ascending=False).astype(int)
colorscale = [[0, '#800000'], [.5, '#fffffe'], [1, '#ffffff']]
fig = ff.create_table(Result_2020[['Rank', 'Player', 'Original_Draft_Pick', 'WS_Pred', 'Rank_of_Current_WS']]
                     .sort_values('WS_Pred', ascending=False)
                     .head(15), colorscale)
for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i].font.size = 13
fig.show()

###Visualizations code for top 15 WS: Change Year of df name for year wanted to view. (2020)
Result_2020['Rank'] = Result_2020['WS'].rank(method='dense', ascending=False).astype(int)
colorscale = [[0, '#800000'], [.5, '#fffffe'], [1, '#ffffff']]
fig = ff.create_table(Result_2020[['Rank', 'Player', 'Original_Draft_Pick', 'Current_WS', 'Rank_of_WS_Prediction']]
                     .sort_values('Current_WS', ascending=False)
                     .head(15), colorscale)
for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i].font.size = 13
fig.show()


Training R^2: 0.867508152571651
Testing R^2: -8.129951458628783
